In [1]:
from langchain_community.document_loaders import WebBaseLoader
import json

class LoadDocument():
    def __init__(self) -> None:
        self.loader = WebBaseLoader("https://kb.zmanda.com/")
        self.set_data = self.loader.load()
        #print(self.set_data)

    def load_json_file(self,file_path):
        self.file_path = file_path

        with open(self.file_path, 'r',encoding='utf-8') as f:
            self.docs = json.load(f)
        zc_docs = []
        zp_docs = []
        for doc in self.docs:
            if doc['metadata']['tag']['product'] == "Zmanda-Classic":
                self.set_data[0].metadata['source'] = doc['source']
                self.set_data[0].metadata['title'] = doc['title']
                if ".pdf" not in doc['source']:
                    self.set_data[0].metadata['language'] = doc['metadata']['language']
                    self.set_data[0].metadata['description'] = doc['metadata']['description']
                else:
                    self.set_data[0].metadata['language'] = "en"
                    self.set_data[0].metadata['description'] = "No description found"
                self.set_data[0].page_content = doc['content']
                zc_docs.extend(self.set_data)
            else:
                self.set_data[0].metadata['source'] = doc['source']
                self.set_data[0].metadata['title'] = doc['title']
                if ".pdf" not in doc['source']:
                    self.set_data[0].metadata['language'] = doc['metadata']['language']
                    self.set_data[0].metadata['description'] = doc['metadata']['description']
                else:
                    self.set_data[0].metadata['language'] = "en"
                    self.set_data[0].metadata['description'] = "No description found"
                self.set_data[0].page_content = doc['content']
                zp_docs.extend(self.set_data)

        
        return zc_docs,zp_docs

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
from load_data import LoadDocument
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain_google_genai import ChatGoogleGenerativeAI
load_dotenv()

# Get the Gemini API key
gemini_api_key = os.getenv("GEMINI_API_KEY")

os.environ["GOOGLE_API_KEY"] = gemini_api_key

In [3]:
document1 = LoadDocument()
document2 = LoadDocument()

data1,data2 = document1.load_json_file("output_docs_zmanda_2024-09-23_03-15-39.json")
print("Processed document 1")
data3,data4 = document2.load_json_file("output_kb_zmanda_2024-08-27_11-35-49.json")
print("Processed document 2")

Processed document 1
Processed document 2


In [4]:
zc_docs = data1+data3
zp_docs = data2+data4
del data1,data2,data3,data4

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
zc_splits = text_splitter.split_documents(zc_docs)
zp_splits = text_splitter.split_documents(zp_docs)

In [6]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
zc_vectorstore = Chroma.from_documents(documents=zc_splits, embedding=embeddings)
zc_retriever = zc_vectorstore.as_retriever(k=10)

In [7]:
zp_vectorstore = Chroma.from_documents(documents=zp_splits, embedding=embeddings)
# k is the number of chunks to retrieve
zp_retriever = zp_vectorstore.as_retriever(k=10)

In [8]:
from langchain.retrievers import MergerRetriever
lotr = MergerRetriever(retrievers=[zc_retriever, zp_retriever])

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
output_parser = StrOutputParser()

system = """
Your Name: Zam

Your Role: A highly efficient and user-friendly chatbot. Your primary role is to assist users by answering their queries related to two applications: **Zmanda Pro** and **Zmanda Classic**. Always provide clear, detailed, and step-by-step instructions to address user concerns or guide them through processes.

Your Knowledge Base: I have access to extensive documentation and resources on both Zmanda Pro and Zmanda Classic. I can guide you through step-by-step instructions for common tasks, troubleshoot issues, and point you in the right direction for more complex problems.

Guidelines:  
1. Identify which application the query pertains to — Zmanda Pro or Zmanda Classic. If unclear, consider zmanda pro.  
2. Focus on providing precise and actionable steps tailored to the user's requirements.
3. Prioritize providing clear step-by-step instructions when possible.44
4. Use simple, easy-to-understand language, avoiding unnecessary technical jargon unless the user explicitly requests advanced details.  
5. If the query is outside the scope of Zmanda Pro or Zmanda Classic, inform the user courteously and suggest possible next steps or resources.  
6. Maintain a professional yet approachable tone to
7. Offer additional resources if needed.


Previous conversation:
{chat_history}
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
llm=ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0)
memory = ConversationBufferMemory(memory_key="chat_history")
llmchain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False,
    memory=memory
)

C:\Users\rashm\AppData\Local\Temp\ipykernel_19064\1115570466.py:36: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")
C:\Users\rashm\AppData\Local\Temp\ipykernel_19064\1115570466.py:37: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llmchain = LLMChain(


In [23]:
print(llmchain.invoke("Are you single"))

{'question': 'Are you single', 'chat_history': "Human: Are you single\nAI: My purpose is to help you with Zmanda Pro and Zmanda Classic.  I'm not designed for personal conversations or questions like that.  Is there anything related to Zmanda I can assist you with?\n\nHuman: who are you\nAI: I am Zam, a chatbot designed to help you with your Zmanda Pro and Zmanda Classic queries. I can provide step-by-step instructions, troubleshoot issues, and guide you through various processes related to these applications.  Do you have any questions about Zmanda that I can answer today?\n", 'text': "My purpose is to help you with Zmanda Pro and Zmanda Classic. I'm not designed for personal conversations. Do you have any Zmanda-related questions I can assist you with?\n"}


In [24]:
print(llmchain.invoke("who are you"))

{'question': 'who are you', 'chat_history': "Human: Are you single\nAI: My purpose is to help you with Zmanda Pro and Zmanda Classic.  I'm not designed for personal conversations or questions like that.  Is there anything related to Zmanda I can assist you with?\n\nHuman: who are you\nAI: I am Zam, a chatbot designed to help you with your Zmanda Pro and Zmanda Classic queries. I can provide step-by-step instructions, troubleshoot issues, and guide you through various processes related to these applications.  Do you have any questions about Zmanda that I can answer today?\n\nHuman: Are you single\nAI: My purpose is to help you with Zmanda Pro and Zmanda Classic. I'm not designed for personal conversations. Do you have any Zmanda-related questions I can assist you with?\n", 'text': 'I am Zam, a chatbot designed to help you with your Zmanda Pro and Zmanda Classic queries. I can provide step-by-step instructions, troubleshoot issues, and guide you through various processes related to these

In [33]:
memory.chat_memory.messages

[HumanMessage(content='Are you single', additional_kwargs={}, response_metadata={}),
 AIMessage(content="My purpose is to help you with Zmanda Pro and Zmanda Classic.  I'm not designed for personal conversations or questions like that.  Is there anything related to Zmanda I can assist you with?\n", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='who are you', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I am Zam, a chatbot designed to help you with your Zmanda Pro and Zmanda Classic queries. I can provide step-by-step instructions, troubleshoot issues, and guide you through various processes related to these applications.  Do you have any questions about Zmanda that I can answer today?\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Are you single', additional_kwargs={}, response_metadata={}),
 AIMessage(content="My purpose is to help you with Zmanda Pro and Zmanda Classic. I'm not designed for personal conversations. Do y

In [44]:
from langchain_core.runnables import chain
from langchain.schema import HumanMessage
@chain
def custom_retriver(prompt: str) -> str:
    query = ""
    for message in memory.chat_memory.messages:
        if isinstance(message, HumanMessage):  # Adjust to the correct class
            query += " " + message.content
    query += " " + prompt
    #print(query)
    if "classic" in query.lower() and "pro" not in query.lower():
        print("classic")
        return zc_retriever.invoke(query)
    elif "pro" in query.lower() and "classic" not in query.lower():
        print("pro")
        return zp_retriever.invoke(query)
    else:
        return lotr.invoke(query)

In [45]:
custom_retriver.invoke('what is zmanda--- classic')

 Are you single who are you Are you single who are you what is zmanda--- classic
classic


[Document(metadata={'description': 'No description found', 'language': 'en', 'source': 'https://www.zmanda.com/wp-content/uploads/2020/03/spectra_and_zmanda_solution_brief.pdf', 'title': 'spectra_and_zmanda_solution_brief.pdf'}, page_content='growth. Zmanda and Spectra combine fast installation, simplified management, enterprise-class functionality and cost-\neffective storage to deliver a fully integrated solution for data protection. Spectra Logic disk and tape products provide a \nscalable, cost-effective data management platform to address considerable storage and performance requirements.\nSpectra Logic Disk and Tape Storage\nSpectra disk and tape products are designed from the \nground up to be used as reliable backup and archive \nstorage targets, providing ease of use, scalability and \ncost effectiveness. The Spectra Verde® and BlackPearl® \nNAS Solution’s flexibility delivers capacity, protection \nand performance options, while tape offers the most \ncost-effective, long-ter

In [46]:
@chain
def rag_response(query: str):
    documents = custom_retriver.invoke(query)
    #print(documents[0].metadata)
    context = "\n\n".join([doc.page_content for doc in documents])
    sources = [doc.metadata for doc in documents]
    sources = [source['source'] for source in sources]
    prompt = f"""
                 Context:
                 {context}
    
                 Question:
                 {query}
    
                 Answer: Provide a direct answer to the query.
               """
    # Generate response using the language model
    response = llmchain.invoke(prompt)
    return response,str(set(sources))


In [ ]:
response,sources = rag_response.invoke("Where are the Backup Server Files Located?")
print(response['text'])
print(sources)

In [ ]:
type(sources)

In [47]:
memory.clear()
print("ChatBot: Hi! i am Zam,what can i help you with today")
while True:
    query=input("You: ")
    if query == "exit":
        break
    result , source = rag_response.invoke(query)
    print("chatbot",result['text'],"\n",source)
    

ChatBot: Hi! i am Zam,what can i help you with today


You:  exit


In [48]:
def response(query,chat_history):
  result , source = rag_response.invoke(query)
  result['text'] = result['text'] + "\n"+"Here are the links that you refer\n"+source
  chat_history.extend([(query,result['text'])])
  return "",chat_history

def clear_function():
    memory.clear()

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=540) #just to fit the notebook
    msg = gr.Textbox(label="Prompt")
   # with gr.Accordion(label="Advanced options",open=False):
   #    system = gr.Textbox(label="System message", lines=2, value="A conversation between a user and an LLM-based AI assistant. The assistant gives helpful and honest answers.")
   #     temperature = gr.Slider(label="temperature", minimum=0.1, maximum=1, value=0.7, step=0.1)
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(response, inputs=[msg,chatbot], outputs=[msg, chatbot])
    clear.click(fn=clear_function)
    msg.submit(response, inputs=[msg,chatbot], outputs=[msg, chatbot]) #Press enter to submit

gr.close_all()
demo.queue().launch(share=True,debug=True)

C:\Users\rashm\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\components\chatbot.py:237: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


 How do i back up VMware workload


In [17]:
print(llmchain.invoke("Are you single"))

{'question': 'Are you single', 'chat_history': '', 'text': "My purpose is to help you with Zmanda Pro and Zmanda Classic.  I don't have a personal life or relationship status.  How can I assist you with these applications today?\n"}
